In [1]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import random
from hmmlearn import hmm
from hmmlearn.base import ConvergenceMonitor
import plotly.express as px

In [2]:
df = pd.read_excel('dl_df_deposit.xlsx')
indicator_data = df.set_index("INDICATOR")

time_series_data = indicator_data.values 
print("Количество временных рядов:", time_series_data.shape[0])  
print("Длина каждого временного ряда:", time_series_data.shape[1]) 

# Нормализуем данные
time_series_data = TimeSeriesScalerMeanVariance().fit_transform(time_series_data)

Количество временных рядов: 35
Длина каждого временного ряда: 197


### С помощью метода локтя определяем оптимальное колво кластеров

In [ ]:
inertia = []  # Для хранения значений инерции
k_values = range(1, 11)  # Количество кластеров для проверки

# Проходим по количествам кластеров
for k in k_values:
    model = TimeSeriesKMeans(n_clusters=k, metric="dtw", verbose=False, random_state=42)
    model.fit(time_series_data)
    inertia.append(model.inertia_)  # Добавляем инерцию для текущего k

# Строим график метода локтя
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia, 'bo-', linewidth=2)
plt.title('Метод локтя для временных рядов (DTW)', fontsize=14)
plt.xlabel('Количество кластеров', fontsize=12)
plt.ylabel('Инерция', fontsize=12)
plt.grid()
plt.show()


### Кластеризируем с помощью TKMeans

In [ ]:
# Кластеризация
n_clusters = 4
model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", verbose=True, random_state=42)
clusters = model.fit_predict(time_series_data)

# Создаём DataFrame с результатами кластеризации
indicator_clusters = pd.DataFrame({
    "Indicator": indicator_data.index,  # Индикаторы теперь это индексы
    "Cluster": clusters
})

# Выводим распределение индикаторов по кластерам
print(indicator_clusters.groupby("Cluster").size())


### Смотрим на корректность разбиения

In [ ]:
plt.figure(figsize=(20, 10))
for cluster in range(n_clusters):
    plt.subplot(2, 5, cluster + 1)
    for i, series in enumerate(time_series_data):
        if clusters[i] == cluster:
            plt.plot(series.ravel(), alpha=0.5)
    plt.title(f'Кластер {cluster}')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
for cluster_idx in range(n_clusters):
    plt.plot(model.cluster_centers_[cluster_idx].ravel(), label=f'Кластер {cluster_idx}')
plt.title('Центроиды кластеров')
plt.xlabel('Временные точки')
plt.ylabel('Значения')
plt.legend()
plt.grid()
plt.show()


In [ ]:
indicator_clusters

### Добавляем в датасет метки кластеров

In [ ]:
df = pd.read_excel('dl_df_deposit.xlsx')
indicator_data = df.set_index("INDICATOR") 
time_series_data = indicator_data.values 
scaler = TimeSeriesScalerMeanVariance()
normalized_data = scaler.fit_transform(time_series_data)
normalized_df = pd.DataFrame(normalized_data.reshape(normalized_data.shape[0], -1), 
                              index=indicator_data.index, 
                              columns=indicator_data.columns)
normalized_df = normalized_df.reset_index()
result = pd.merge(normalized_df, indicator_clusters, left_on='INDICATOR', right_on='Indicator', how='inner')
result = result.drop(columns=['Indicator',"INDICATOR"])
result = result.set_index(['Cluster'])
result

### Проходимся PCA по каждому кластеру в каждый момент времени

In [ ]:
# Получаем уникальные кластеры и месяцы
clusters = result.index.get_level_values(0).unique()  
months = result.columns 
pca_results = []

for cluster in clusters:
    cluster_data = result.loc[cluster] 
    
    for month in months:
        month_data = cluster_data[month].dropna().values.reshape(-1, 1)  # Извлекаем данные за месяц и удаляем пропуски
        if month_data.size > 0:
            pca = PCA(n_components=1) 
            pca_result = pca.fit_transform(month_data)  
            pca_results.append({
                'Cluster': cluster,
                'Month': month,
                'PCA_Result': pca_result.flatten()[0]
            })

results_df = pd.DataFrame(pca_results)
print(results_df)

In [ ]:
pivot_df = results_df.pivot(index='Month', columns='Cluster', values='PCA_Result')
pivot_df

### Строим модель

In [ ]:
# Для воспроизводимости графика
random.seed(42)
np.random.seed(42)

pivot_df = results_df.pivot(index='Month', columns='Cluster', values='PCA_Result')
X = pivot_df.values
model = hmm.GaussianHMM(n_components=4, covariance_type="full", init_params="mc", n_iter=1000, tol=0.001, random_state=42) 
model.startprob_ = np.array([0, 1, 0, 0]) 

#Предполагаем, что вероятность перейти в другое состояние маленькая
# Начальные значения для p_00, p_11, p_22 и p_33
p_00 = 0.9   # Вероятность остаться в состоянии 0
p_11 = 0.9   # Вероятность остаться в состоянии 1
p_22 = 0.9   # Вероятность остаться в состоянии 2
p_33 = 0.9   # Вероятность остаться в состоянии 3

# Матрица переходов, с учетом, что из пика нельзя попасть сразу в рецессию итд
model.transmat_ = np.array([
    [p_00, (1 - p_00), 0, 0],          # Переходы из состояния 0
    [0, p_11, (1 - p_11), 0],          # Переходы из состояния 1
    [0, 0, p_22, (1 - p_22)],          # Переходы из состояния 2
    [(1 - p_33), 0, 0, p_33]           # Переходы из состояния 3
])

model.fit(X)
hidden_states = model.predict(X)

# Вывод начальных параметров
print("Начальные вероятности:")
print(model.startprob_)
print("Начальная матрица переходов:")
print(model.transmat_)
print("Начальные средние значения:")
print(model.means_)
print("Начальные ковариационные матрицы:")
print(model.covars_)

print("Обученная матрица переходов:")
print(model.transmat_)
print("Изменения лосса на каждом шаге:")
model.monitor_

Сохраняем веса модели

In [ ]:
import pickle
initial_params = {
    'startprob_': model.startprob_,
    'transmat_': model.transmat_,
    'means_': model.means_,
    'covars_': model.covars_
}

with open('initial_params.pkl', 'wb') as f:
    pickle.dump(initial_params, f)
with open('initial_params.pkl', 'rb') as f:
    loaded_params = pickle.load(f)
print("Загруженные начальные параметры:")
print("Начальные вероятности:")
print(loaded_params['startprob_'])
print("Начальная матрица переходов:")
print(loaded_params['transmat_'])
print("Начальные средние значения:")
print(loaded_params['means_'])
print("Начальные ковариационные матрицы:")
print(loaded_params['covars_'])

In [ ]:
history = list(model.monitor_.history)  
plt.figure(figsize=(10, 6))
plt.plot(history, marker='o')
plt.title('История логарифмической вероятности')
plt.xlabel('Итерация')
plt.ylabel('Логарифмическая вероятность')
plt.grid()
plt.show()

In [ ]:
log_likelihood = model.score(X)
print(f"Логарифмическая вероятность наблюдаемых данных: {log_likelihood}")

In [ ]:
pivot_df['Hidden_States'] = hidden_states
pivot_df['Hidden_States'] = pivot_df['Hidden_States'].replace({0:0.1, 1:1, 2: -0.1, 3:-1}) #меняем значения для красивого графика
pivot_df = pivot_df.reset_index()
pivot_df['Hidden_States_Label'] = pivot_df['Hidden_States'].replace({0.1: 'рост', 1: 'пик', -0.1: 'спад', -1: 'рецессия'})
pivot_df = pivot_df.reset_index()

### Результат моделирования

In [ ]:
fig = px.line(pivot_df, x='Month', y='Hidden_States', title='Интерактивный график Hidden States', hover_data={'Hidden_States_Label': True})
fig.show()

### Дополнительный график

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pivot_df['Month'], pivot_df['Hidden_States'], marker='o', linestyle='-', color='b')
plt.title('График Hidden States')
plt.xlabel('Месяц')
plt.ylabel('Hidden States')
plt.show()

### Дополнительный код, для запуска модели с сохраненными весами 

In [ ]:
import pickle
from hmmlearn import hmm

with open('initial_params.pkl', 'rb') as f:
    loaded_params = pickle.load(f)

model = hmm.GaussianHMM(n_components=4, covariance_type="full", init_params="mc", n_iter=1000, tol=0.001, random_state=42)

# Установка загруженных параметров
model.startprob_ = loaded_params['startprob_']
model.transmat_ = loaded_params['transmat_']
model.means_ = loaded_params['means_']
model.covars_ = loaded_params['covars_']

hidden_states = model.predict(X)
print("Обученные параметры:")
print("Начальные вероятности:")
print(model.startprob_)
print("Матрица переходов:")
print(model.transmat_)
print("Средние значения:")
print(model.means_)
print("Ковариационные матрицы:")
print(model.covars_)

In [ ]:
pivot_df = results_df.pivot(index='Month', columns='Cluster', values='PCA_Result')
pivot_df['Hidden_States'] = hidden_states
pivot_df['Hidden_States'] = pivot_df['Hidden_States'].replace({0:0.1, 1:1, 2: -0.1, 3:-1}) #меняем значения для красивого графика
pivot_df = pivot_df.reset_index()
pivot_df['Hidden_States_Label'] = pivot_df['Hidden_States'].replace({0.1: 'рост', 1: 'пик', -0.1: 'спад', -1: 'рецессия'})
pivot_df = pivot_df.reset_index()
fig = px.line(pivot_df, x='Month', y='Hidden_States', title='Интерактивный график Hidden States', hover_data={'Hidden_States_Label': True})
fig.show()

### Дополнительный график

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pivot_df['Month'], pivot_df['Hidden_States'], marker='o', linestyle='-', color='b')
plt.title('График Hidden States')
plt.xlabel('Месяц')
plt.ylabel('Hidden States')
plt.show()